In [ ]:
%load_ext jupyternotify

In [1]:
import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm
import my_process_funcs as prc # new

In [2]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
#modelpath = filepath + "004_GB_IEEE123_OPAL_accur.xlsx"
modelpath = filepath + "13NFunbal_impedance.xlsx"

#==========================================================================================================

#file_name = string specifying name of dot file created when make_graph() is called
file_name = '13NFunbalanced'
#file_name = 'demo123NF.dot'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_150'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

2022-06-14 15:11:12.212981


In [3]:
# [ESSENTIAL] create feeder object

fin_feeder = ff.feeder_init(modelpath,'','',timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, depths,file_name)

#print('depths=',depths) # check this is populated, lists how far each node is from substation
#print('depths length=',len(depths))

# print list of first 10 buses in network
print("First ten nodes are:")
count = 0 
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
import csv  # jaimie do you need this?
graph = fin_feeder.network

Finished initializing feeder
n= 12
First ten nodes are:
bus_611
bus_632
bus_633
bus_634
bus_645
bus_646
bus_650
bus_651
bus_652
bus_671


In [4]:
# print z_to_sub for all buses to a csv
graphNodes_nosub = hm.remove_subst_nodes(fin_feeder, file_name) # dont consider substation nodes, node 650 and 651 for 13NF

myabsZs=np.array([], dtype=np.int64).reshape(0,3)

for busName in graphNodes_nosub: # for all non-slack buses
    z2sub=np.around(imp.get_total_impedance_from_substation(fin_feeder, busName,depths),2)
    zself=[[z2sub[0][0], z2sub[1][1], z2sub[2][2]]]
    print('zzelf=',zself)
    abs_z=np.absolute(zself)
    print('abs_z=',abs_z)
    myabsZs=np.append(myabsZs,np.array(abs_z),axis=0) 

print(myabsZs)
#print(np.array(myabsZs))
np.savetxt('z2sub2_13NFunbal.csv', np.array(myabsZs), delimiter=',')

zzelf= [[(0.34+0.85j), (0.26+0.79j), (0.41+0.94j)]]
abs_z= [[0.91547802 0.83168504 1.02552426]]
zzelf= [[(0.13+0.39j), (0.13+0.4j), (0.13+0.39j)]]
abs_z= [[0.4110961  0.42059482 0.4110961 ]]
zzelf= [[(0.2+0.5j), (0.2+0.51j), (0.2+0.51j)]]
abs_z= [[0.53851648 0.54781384 0.54781384]]
zzelf= [[(0.2+0.5j), (0.2+0.51j), (0.2+0.51j)]]
abs_z= [[0.53851648 0.54781384 0.54781384]]
zzelf= [[(0.13+0.39j), (0.25+0.53j), (0.26+0.52j)]]
abs_z= [[0.4110961  0.58600341 0.58137767]]
zzelf= [[(0.13+0.39j), (0.33+0.6j), (0.33+0.6j)]]
abs_z= [[0.4110961  0.68476273 0.68476273]]
zzelf= [[(0.54+0.93j), (0.26+0.79j), (0.33+0.86j)]]
abs_z= [[1.0754069  0.83168504 0.9211406 ]]
zzelf= [[(0.26+0.77j), (0.26+0.79j), (0.26+0.78j)]]
abs_z= [[0.81271151 0.83168504 0.82219219]]
zzelf= [[(0.34+0.81j), (0.33+0.83j), (0.33+0.83j)]]
abs_z= [[0.87846457 0.89319651 0.89319651]]
zzelf= [[(0.33+0.96j), (0.32+0.99j), (0.32+0.98j)]]
abs_z= [[1.01513546 1.0404326  1.03092192]]
zzelf= [[(0.34+0.85j), (0.26+0.79j), (0.33+0.86j)]]

In [5]:
# write busnames into a csv
import csv
graphNodes_noSub=hm.remove_subst_nodes(fin_feeder, file_name) # remove substation nodes, busList will have as many entries as R and X matrix length
assert(len(graphNodes_noSub)*3==len(R)) # *3 is because R is 3ph
with open("13NFunbal_busList.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='-',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(graphNodes_noSub)

In [6]:
# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
np.savetxt('Rmat_13NFunbal_accur.csv', R, delimiter=',')
np.savetxt('Xmat_13NFunbal_accur.csv', X, delimiter=',')